In [ ]:
import numpy as np

In [ ]:
def find_max_rank( ranks ):
    """ Find the highest ranked item """
    max_rank  = 0
    for i in len(ranks):
        if ranks[i] > max_rank:
            max_rank = ranks[i]
    return max_rank

In [ ]:
def find_max_rating_score(user_item_ratings):
    max_item_score = 0 
    
    for i in range(len(user_item_ratings)):
        for j in range(len(user_item_ratings[i])):
                if user_item_ratings[i][j] > max_item_score:
                    max_item_score = user_item_ratings[i][j]
    return max_item_score

$$ ||u|| = \sqrt{u_{0}^{2}+u_{1}^{2}......u_{n}^{2}} $$

In [ ]:
def norm_vector(vector):
    output = 0
    for i in len(vector):
        output = output + ( vector[i] ** 2 ) 
    return np.sqrt(output)

$$ {U_{i}^{T}\cdot V_{j}} $$

In [ ]:
def user_feature_dot_item_feature(user_feature_vector, item_feature_vector):
    return np.dot(np.transpose(user_feature_vector), item_feature_vector)

$$ || U_{i} || \cdot || V_{j} || $$

In [ ]:
def norm_user_feature_dot_norm_item_feature(user_feature_vector, item_feature_vector):
    return np.dot(norm_vector(user_feature_vector), norm_vector(item_feature_vector))

$$ \frac{{U_{i}^{T}\cdot V_{j}}}{ || U_{i} || \cdot || V_{j} || } $$

In [ ]:
def user_dot_item_div_norm(user_feature_vector, item_feature_vector):
    user_dot_item = user_feature_dot_item_feature(user_feature_vector, item_feature_vector)
    norm_user_dot_item = norm_user_feature_dot_norm_item_feature(user_feature_vector, item_feature_vector)
    return np.divide(user_dot_item, norm_user_dot_item)

#### Estimate Rank Function

$$ x_{j} = \sum \limits _{i=1} ^{m} \alpha_{i}U_{i}^{T}\cdot V_{j} $$

In [ ]:
def estimate_rank( alpha_coefficents, user_feature_vectors, item_feature_vector ):
    """Calculate the estimated rank of the item based on the user_feature_vectors and the item_feature_vector"""
    rank = 0

    for i in range( 0, len(user_feature_vectors)):
        rank = rank + np.multiply(alpha_coefficents[i], np.dot(user_feature_vectors[i], item_feature_vector))
    return rank

#### Zipf Exponent

$$ s = 1 + n \left( \begin{array}{cc} \sum \limits _{i=1} ^{n} ln \frac{x_{i}}{x_{max}}  \end{array}\right)^{-1}  $$

In [ ]:
def zipf_exponent(alpha_coefficents, user_feature_vectors, item_feature_vectors):
    ranks = []
    for i in range( item_feature_vectors ):
        ranks.append(estimate_rank(alpha_coefficents, user_feature_vectors, item_feature_vectors[i]))
    max_rank = find_max_rank(ranks)

    sum_ranks = 0
    for j in len(ranks):
        sum_ranks = sum_ranks + np.log(np.divide(ranks[j], max_rank))

    return 1 + np.multiply(len(item_feature_vectors), np.power(sum_ranks, -1))

### Loss Function

$$ L =\sum \limits _{i=1} ^{m}\sum \limits _{j=1} ^{n} \left( \begin{array}{cc} \frac{R_{ij}}{R_{max}} - \frac{U_{i}^{T}\cdot V_{j}}{ || U_{i} || \cdot || V_{j} || } \end{array}\right)^{2} - \beta \left( \begin{array}{cc} 1 + n \left( \begin{array}{cc} \sum \limits _{i=1} ^{n} ln \frac{x_{i}}{x_{max}}  \end{array}\right)^{-1} \end{array}\right) $$

In [ ]:
def loss_function(user_items_ratings_scores, user_feature_vectors, item_feature_vectors, alpha_coefficents,
                  beta_coefficent):
    max_item_score = find_max_rating_score(user_items_ratings_scores)

    matrix_loss = 0
    for i in range(m):
        for j in range(n):
            matrix_loss = matrix_loss + np.divide(user_items_ratings_scores[i][j], max_item_score) - \
                            user_dot_item_div_norm(user_feature_vectors[i], item_feature_vector[j])
                          
    matrix_loss = np.power(matrix_loss, 2)
    
    #Penalize by ZipF exponent
    matrix_loss = matrix_loss - np.multiply(beta_coefficent, zipf_exponent(alpha_coefficents, user_feature_vectors,
                                                                           item_feature_vectors))
    
    return matrix_loss


### Optimal User Feature Vector

$$ U_{i} = U_{i} - \mathcal{E} \cdot \left( \begin{array}{cc} \beta \cdot  \frac{n}{\left( \begin{array}{cc} log\left( \begin{array}{cc} \alpha_{ij} \cdot \frac{t_{0}}{n} \end{array}\right)^{2} \end{array}\right)\cdot t_{0}} \cdot V_{j}
 -2 \cdot  \frac{\left( \begin{array}{cc} R_{ij} - \frac{t_{0}}{t_{3}} \end{array}\right)}{t_{3}} \cdot V_{j} -2 \cdot t_{0} \cdot \frac{\left( \begin{array}{cc}R_{ij}-\frac{t_{o}}{t_{3}}\end{array}\right)}{\left( \begin{array}{cc} t_{1}^{3} \cdot t_{2} \end{array}\right)} \cdot U_{i}  \end{array}\right)$$
 <br>
 where
 $$ t_{0} = U_{i}^{T} \cdot V_{j} $$ 
 <br>
 $$ t_{1} = || U_{i} || $$
 <br>
 $$ t_{2} = || V_{j} || $$
 <br>
 $$ t_{3} = t_{1} \cdot t_{2} $$ 

#### Decompose Matrix Factorizaton

$$  log\left( \begin{array}{cc} \alpha_{ij} \cdot \frac{t_{0}}{n} \end{array}\right)^{2}$$
<br>
Expanded Form:
<br>
$$  log\left( \begin{array}{cc} \alpha_{ij} \cdot \frac{U_{i}^{T} \cdot V_{j}}{n} \end{array}\right)^{2}$$

In [ ]:
def log_alpha_dot_vectors(alpha_coefficent, user_feature_vector, item_feature_vector, n):
    vectors_divide = np.divide(user_feature_dot_item_feature(user_feature_vector, item_feature_vector), n) 
    coeff = np.dot(alpha_coefficent, vectors_divide)
    coeff_log = np.log10(coeff)
    squared_log = np.power(coeff_log, 2)
    return squared_log

$$ \left( \begin{array}{cc} log\left( \begin{array}{cc} \alpha_{ij} \cdot \frac{t_{0}}{n} \end{array}\right)^{2} \end{array}\right)\cdot t_{0}$$
<br>
Expanded Form:
<br>
$$ \left( \begin{array}{cc} log\left( \begin{array}{cc} \alpha_{ij} \cdot \frac{U_{i}^{T} \cdot V_{j}}{n} \end{array}\right)^{2} \end{array}\right)\cdot \left( \begin{array}{cc} U_{i}^{T} \cdot V_{j}\end{array}\right)$$

In [ ]:
def log_square_dot_product(alpha_coefficent, user_feature_vector, item_feature_vector, n):
    dot_products = user_feature_dot_item_feature(user_feature_vector, item_feature_vector)
    log_squared = log_alpha_dot_vectors(alpha_coefficent, user_feature_vector, item_feature_vector, n)
    return np.dot(log_squared, dot_products)

$$ \left( \begin{array}{cc} \beta \cdot  \frac{n}{\left( \begin{array}{cc} log\left( \begin{array}{cc} \alpha_{ij} \cdot \frac{t_{0}}{n} \end{array}\right)^{2} \end{array}\right)\cdot t_{0}} 
  \end{array}\right)$$
  <br>
Expanded Form:
<br>
$$ \left( \begin{array}{cc} \beta \cdot  \frac{n}{\left( \begin{array}{cc} log\left( \begin{array}{cc} \alpha_{ij} \cdot \frac{U_{i}^{T} \cdot V_{j}}{n} \end{array}\right)^{2} \end{array}\right)\cdot U_{i}^{T} \cdot V_{j}}  
  \end{array}\right)$$

In [ ]:
def beta_alpha_squared_dot_vectors(alpha_coefficent, beta_coefficent, user_feature_vector, item_feature_vector, n):
    log_square_dot_product_val = log_square_dot_product(alpha_coefficent, user_feature_vector, item_feature_vector, n )
    log_square_dot_product_val = np.divide(n, log_square_dot_product_val)
    return np.dot(beta_coefficent, item_dot_log_square)

$$ \left( \begin{array}{cc} \beta \cdot  \frac{n}{\left( \begin{array}{cc} log\left( \begin{array}{cc} \alpha_{ij} \cdot \frac{t_{0}}{n} \end{array}\right)^{2} \end{array}\right)\cdot t_{0}} \cdot V_{j}
  \end{array}\right)$$
  <br>
Expanded Form:
<br>
$$ \left( \begin{array}{cc} \beta \cdot  \frac{n}{\left( \begin{array}{cc} log\left( \begin{array}{cc} \alpha_{ij} \cdot \frac{U_{i}^{T} \cdot V_{j}}{n} \end{array}\right)^{2} \end{array}\right)\cdot U_{i}^{T} \cdot V_{j}} \cdot V_{j}
  \end{array}\right)$$

In [ ]:
def beta_alpha_squared_dot_vectors_dot_item(alpha_coefficent, beta_coefficent, user_feature_vector, item_feature_vector, n):
    log_square_dot_product_val = beta_alpha_squared_dot_vectors(alpha_coefficent, beta_coefficent, user_feature_vector, item_feature_vector, n)
    return np.dot(log_square_dot_product_val, item_feature_vector)

$$ \left( \begin{array}{cc} R_{ij} - \frac{t_{0}}{t_{3}} \end{array}\right) $$
<br>
Expanded Form:
<br>
$$\left( \begin{array}{cc} R_{ij} - \frac{ U_{i}^{T} \cdot V_{j}}{|| U_{i} || \cdot || V_{j} ||} \end{array}\right)$$

In [ ]:
def rating_minus_user_item_dot(user_item_rating, user_feature_vector, item_feature_vector):
    user_dot_item_div = user_dot_item_div_norm(user_feature_vector, item_feature_vector)
    rating_minus_user_dot_item = np.subtract(user_item_rating, user_dot_item_div)
    return rating_minus_user_dot_item

$$ \frac{\left( \begin{array}{cc} R_{ij} - \frac{t_{0}}{t_{3}} \end{array}\right)}{t_{3}} $$
<br>
Expanded Form:
<br>
$$ \frac{\left( \begin{array}{cc} R_{ij} - \frac{ U_{i}^{T} \cdot V_{j}}{t_{1} \cdot t_{2}} \end{array}\right)}{t_{1} \cdot t_{2}} $$
<br>Further expansion:
<br>
$$ \frac{\left( \begin{array}{cc} R_{ij} - \frac{ U_{i}^{T} \cdot V_{j}}{|| U_{i} || \cdot || V_{j} ||} \end{array}\right)}{|| U_{i} || \cdot|| V_{j} ||} $$


In [ ]:
def rating_dot_user_item_vectors_product(user_item_rating, user_feature_vector, item_feature_vector):
    user_dot_item_div = rating_minus_user_item_dot(user_item_rating, user_feature_vector, item_feature_vector)
    norms = norm_user_feature_dot_norm_item_feature(user_feature_vector, item_feature_vector)
    return np.divide(rating_minus_user_dot_item, norms)

$$  \left( \begin{array}{cc} 
 2 \cdot  \frac{\left( \begin{array}{cc} R_{ij} - \frac{t_{0}}{t_{3}} \end{array}\right)}{t_{3}} \cdot V_{j} \end{array}\right)$$
 <br>Further expansion:
<br>
$$  
 2 \cdot  \frac{\left( \begin{array}{cc} R_{ij} - \frac{ U_{i}^{T} \cdot V_{j}}{|| U_{i} || \cdot || V_{j} ||} \end{array}\right)}{|| U_{i} || \cdot|| V_{j} ||} \cdot V_{j} $$

In [ ]:
def rating_user_item_product(user_item_rating, user_feature_vector, item_feature_vector):
    rating_dot_user_item_product_val = rating_dot_user_item_vectors_product(user_item_rating, user_feature_vector, item_feature_vector)
    rating_dot_user_item_product_val = np.dot(rating_dot_user_item_vectors_product_val, item_feature_vector)
    rating_dot_user_item_product_val = np.dot(2, rating_dot_user_item_product_val)
    return rating_dot_user_item_product_val

$$ \left( \begin{array}{cc} || U_{i} ||^{3} \cdot || V_{j} || \end{array}\right) $$

In [ ]:
def user_cubed_dot_item(user_feature_vector, item_feature_vector):
    user_norm = norm_vector(user_feature_vector)
    item_norm = norm_vector(item_feature_vector)
    user_cubed = np.power(user_norm, 3)
    return np.dot(user_cubed, item_norm)

$$  2 \cdot t_{0} \cdot \frac{\left( \begin{array}{cc}R_{ij}-\frac{t_{o}}{t_{3}}\end{array}\right)}{\left( \begin{array}{cc} t_{1}^{3} \cdot t_{2} \end{array}\right)} \cdot U_{i} $$
<br>
Expanded:
<br>
$$ 2 \cdot \left( \begin{array}{cc} U_{i}^{T} \cdot V_{j} \end{array}\right) \cdot \frac{ \left( \begin{array}{cc} R_{ij} - \frac{ U_{i}^{T} \cdot V_{j}}{|| U_{i} || \cdot || V_{j} ||} \end{array}\right)}{\left( \begin{array}{cc} || U_{i} ||^{3} \cdot || V_{j} || \end{array}\right)} \cdot U_{i} $$ 
<br>

In [ ]:
def ratings_user_cubed_item_dot_product(user_item_rating, user_feature_vector, item_feature_vector):
    rating_minus_user_item = rating_minus_user_item_dot(user_item_rating, user_feature_vector, item_feature_vector)
    user_cubed = user_cubed_dot_item(user_feature_vector, item_feature_vector)
    rating_div_user_cubed = np.divide(rating_minus_user_item, user_cubed)
    
    user_dot_item = user_feature_dot_item_feature(user_feature_vector, item_feature_vector)
    
    user_dot_rating_cubed = np.dot(user_dot_item, rating_div_user_cubed)
    return np.dot(2, np.dot(user_dot_rating_cubed, user_feature_vector))

$$ U_{i} = U_{i} - \mathcal{E} \cdot \left( \begin{array}{cc} \beta \cdot  \frac{n}{\left( \begin{array}{cc} log\left( \begin{array}{cc} \alpha_{ij} \cdot \frac{t_{0}}{n} \end{array}\right)^{2} \end{array}\right)\cdot t_{0}} \cdot V_{j}
 -2 \cdot  \frac{\left( \begin{array}{cc} R_{ij} - \frac{t_{0}}{t_{3}} \end{array}\right)}{t_{3}} \cdot V_{j} -2 \cdot t_{0} \cdot \frac{\left( \begin{array}{cc}R_{ij}-\frac{t_{o}}{t_{3}}\end{array}\right)}{\left( \begin{array}{cc} t_{1}^{3} \cdot t_{2} \end{array}\right)} \cdot U_{i}  \end{array}\right)$$

In [ ]:
def optimise_user_feature_vector(epsilion, alpha_coefficent, beta_coefficent, user_item_rating, user_feature_vector, item_feature_vector, n):
    beta_alpha =  beta_alpha_squared_dot_vectors_dot_item(alpha_coefficent, beta_coefficent, user_feature_vector, item_feature_vector, n)
    user_rating_product = rating_user_item_product(user_item_rating, user_feature_vector, item_feature_vector)
    rating_cubed = ratings_user_cubed_item_dot_product(user_item_rating, user_feature_vector, item_feature_vector)
    beta_user_rating_product = np.subtract(beta_alpha,user_rating_product)
    beta_user_rating_product_cubed = np.subtract(beta_user_rating_product, rating_cubed)
    epsilion_ratings = np.dot(epsilion, beta_user_rating_product_cubed)
    return np.subtract(user_feature_vector, epsilion_ratings)


### Optimal Item Feature Vector

$$ V_{j} = V_{j} - \mathcal{E} \cdot \left( \begin{array}{cc} \beta \cdot  \frac{n}{\left( \begin{array}{cc} log\left( \begin{array}{cc} \alpha_{ij} \cdot \frac{t_{0}}{n} \end{array}\right)^{2} \end{array}\right)\cdot t_{0}} \cdot U_{i}
-  \frac{t_{4}}{t_{3}} \cdot U_{i} + \frac{\left( \begin{array}{cc}t_{0} \cdot t_{4}\end{array}\right)}{\left( \begin{array}{cc}t_{2}^{3} \cdot t_{1}\end{array}\right)} \cdot V_{j} \end{array}\right)$$
<br>
 where
 $$ t_{0} = U_{i}^{T} \cdot V_{j} $$ 
 <br>
 $$ t_{1} = || U_{i} || $$
 <br>
 $$ t_{2} = || V_{j} || $$
 <br>
 $$ t_{3} = t_{1} \cdot t_{2} $$ 
 <br>
 $$ t_{4} = 2 \cdot \left( \begin{array}{cc} R_{ij} - \frac{t_{0}}{t_{3}} \end{array}\right)$$

### Decompose formulae

$$ \left( \begin{array}{cc} \beta \cdot  \frac{n}{\left( \begin{array}{cc} log\left( \begin{array}{cc} \alpha_{ij} \cdot \frac{t_{0}}{n} \end{array}\right)^{2} \end{array}\right)\cdot t_{0}} \cdot U_{i} \end{array}\right)$$

In [ ]:
def beta_alpha_squared_dot_vectors_dot_user(alpha_coefficent, beta_coefficent, user_feature_vector, item_feature_vector, n):
    beta_alpha = beta_alpha_squared_dot_vectors(alpha_coefficent, beta_coefficent, user_feature_vector, item_feature_vector, n)
    return np.dot(beta_alpha, user_feature_vector)

$$ \frac{t_{4}}{t_{3}}$$
<br>
expanded form:
<br>
$$ \frac{ 2 \cdot \left( \begin{array}{cc} R_{ij} - \frac{U_{i}^{T} \cdot V_{j}}{|| U_{i} || \cdot|| V_{j} ||} \end{array}\right) }{|| U_{i} || \cdot || V_{j} ||} $$

In [ ]:
def t3_divide_t4(user_item_rating, user_feature_vector, item_feature_vector):
    ratings = rating_minus_user_item_dot(user_item_rating, user_feature_vector, item_feature_vector)
    ratings = np.dot(2, ratings)
    norms = norm_user_feature_dot_norm_item_feature(user_feature_vector, item_feature_vector)
    return np.divide( ratings, norm )

$$ \frac{t_{4}}{t_{3}} \cdot U_{i} $$
<br>
expanded:
<br>
$$  \frac{ 2 \cdot \left( \begin{array}{cc} R_{ij} - \frac{U_{i}^{T} \cdot V_{j}}{|| U_{i} || \cdot|| V_{j} ||} \end{array}\right) }{|| U_{i} || \cdot || V_{j} ||} \cdot U_{i} $$

In [ ]:
def t3_divide_t4_dot_user(user_item_rating, user_feature_vector, item_feature_vector):
    t3t4= t3_divide_t4(user_item_rating, user_feature_vector, item_feature_vector)
    return np.dot(t3t4, user_feature_vector)

$$ \left( \begin{array}{cc}t_{0} \cdot t_{4}\end{array}\right)$$
<br>
expanded:
<br>
$$\left( \begin{array}{cc}  U_{i}^{T} \cdot V_{j}\end{array}\right) \cdot  \left( \begin{array}{cc}2 \cdot \left( \begin{array}{cc} R_{ij} - \frac{U_{i}^{T} \cdot V_{j}}{|| U_{i} || \cdot|| V_{j} ||} \end{array}\right) \end{array}\right)  $$

In [ ]:
def t0_dot_t4(user_item_rating, user_feature_vector, item_feature_vector):
    ratings = rating_minus_user_item_dot(user_item_rating, user_feature_vector, item_feature_vector)
    ratings = np.dot(2, ratings)
    user_dot_item = user_feature_dot_item_feature(user_feature_vector, item_feature_vector)
    return np.dot(user_dot_item, ratings)

$$ \left( \begin{array}{cc}t_{2}^{3} \cdot t_{1}\end{array}\right) $$ 
<br>
expanded: 
<br>
$$ \left( \begin{array}{cc}||V_{j}||^{3} \cdot ||U_{i}||\end{array}\right) $$

In [ ]:
def t2_cubed_dot_t1(user_feature_vector, item_feature_vector):
    t2 = norm_vector(item_feature_vector)
    t2_cubed = np.power(t2, 3)
    t1 = norm_vector(user_feature_vector)
    return np.dot(t2_cubed, t1)

$$ \frac{\left( \begin{array}{cc}t_{0} \cdot t_{4}\end{array}\right)}{\left( \begin{array}{cc}t_{2}^{3} \cdot t_{1}\end{array}\right)} \cdot V_{j} $$
<br>
expanded:
<br>
$$\frac{\left( \begin{array}{cc}  U_{i}^{T} \cdot V_{j}\end{array}\right) \cdot  \left( \begin{array}{cc}2 \cdot \left( \begin{array}{cc} R_{ij} - \frac{U_{i}^{T} \cdot V_{j}}{|| U_{i} || \cdot|| V_{j} ||} \end{array}\right) \end{array}\right)}{\left( \begin{array}{cc}||V_{j}||^{3} \cdot ||U_{i}||\end{array}\right)}  $$

In [ ]:
def t0_dot_t4_t2_t1( user_item_rating, user_feature_vector, item_feature_vector ):
    t0_t4 = t0_dot_t4(user_item_rating, user_feature_vector, item_feature_vector)
    t2_3_dot_t1 = t2_cubed_dot_t1(user_feature_vector, item_feature_vector)
    t0_t4_div_t2 = np.divide(t0_t4, t2_3_dot_t1)
    return np.dot(t0_t4_div_t2, item_feature_vector)

$$ V_{j} = V_{j} - \mathcal{E} \cdot \left( \begin{array}{cc} \beta \cdot  \frac{n}{\left( \begin{array}{cc} log\left( \begin{array}{cc} \alpha_{ij} \cdot \frac{t_{0}}{n} \end{array}\right)^{2} \end{array}\right)\cdot t_{0}} \cdot U_{i}
-  \frac{t_{4}}{t_{3}} \cdot U_{i} + \frac{\left( \begin{array}{cc}t_{0} \cdot t_{4}\end{array}\right)}{\left( \begin{array}{cc}t_{2}^{3} \cdot t_{1}\end{array}\right)} \cdot V_{j} \end{array}\right)$$

In [ ]:
def optimise_item_vector(epsilion, alpha_coefficent, beta_coefficent, user_item_rating, user_feature_vector, item_feature_vector, n):
    beta_alpha_dot_user = beta_alpha_squared_dot_vectors_dot_user(alpha_coefficent, beta_coefficent, user_feature_vector, item_feature_vector, n)
    t3_div_t4 = t3_divide_t4_dot_user(user_item_rating, user_feature_vector, item_feature_vector)
    t0_dot_t4_dot_item = t0_dot_t4_t2_t1( user_item_rating, user_feature_vector, item_feature_vector )
    t4_add_t3 = np.add(t3_div_t4, t0_dot_t4_dot_item)
    beta_minus_t3_t4 = np.subtract(beta_alpha_dot_user, t4_add_t3)
    epsilion_dot = np.dot(epsilion, beta_minus_t3_t4)
    return np.subtract(item_feature_vector, epsilion_dot )